## XGB

算法原理  
损失函数    
分裂结点算法  
正则化  
对缺失值处理  
优缺点  
应用场景  
sklearn参数  

## XGB简介

全称：eXtreme Gradient Boosting（极值梯度提升算法）  
作者：陈天奇(华盛顿大学博士)   
基础：GBDT   
所属：boosting迭代型、树类算法。   
适用范围：分类、回归等  
优点：速度快、效果好、能处理大规模数据、支持多种语言、支持自定义损失函数等等。  
xgboost是由梯度提升树GBDT发展而来。梯度提升树可以有回归树和分类树，两者都是以CART树算法为主流，xgboost背后也是CART树，这意味着xgboost中所有树都是二叉树。XGBoost其实是对GBDT算法的一种改良实现，主要包括了正则化、损失函数、学习策略、并行实现等一些方面。

## 算法原理

 GBDT的原理是，首先使用训练集和样本真值（即标准答案）训练一棵树，然后使用这棵树预测训练集，得到每个样本的预测值，由于预测值与真值存在偏差，所以二者相减可以得到“残差”。接下来训练第二棵树，此时不再使用真值，而是使用残差作为标准答案。两棵树训练完成后，可以再次得到每个样本的残差，然后进一步训练第三棵树，以此类推。树的总棵数可以人为指定，也可以监控某些指标（例如验证集上的误差）来停止训练。在预测新样本时，每棵树都会有一个输出值，将这些输出值相加，即得到样本最终的预测值。  
 
 xgboost算法思想就是不断地添加树，不断地进行特征分裂来生长一棵树，每次添加一个树，其实是学习一个新函数，去拟合上次预测的残差。当我们训练完成得到k棵树，我们要预测一个样本的分数，其实就是根据这个样本的特征，在每棵树中会落到对应的一个叶子节点，每个叶子节点就对应一个分数，最后只需要将每棵树对应的分数加起来就是该样本的预测值。


## 损失函数

对于回归问题，常用损失函数是MSE 均方误差  
对于分类问题，常用损失函数是对数损失函数

## 分裂结点算法

精确分裂算法（exact split finding） 精确的贪心法，枚举所有分割点。当数据量十分庞大，以致于不能全部放入内存时，Exact Greedy 算法就会很慢。 

近似分类算法（approximate split finding） 针对精确贪心法的不足，引入了近似的算法。简单来说，就是根据特征k的分布来确定l个候选切分点Sk={sk1,sk2,…,skl}，然后根据这些候选切分点把相应的样本放入对应的桶中，对每个桶中的G,H进行累加。最后在候选切分点集合上贪心查找。  

加权分位法  

稀疏自适应分割策略  

## 正则化

xgboost在目标函数中使用了正则化惩罚项。gama越大，越希望获得结构简单的树，因为此时对较多叶子节点的树的惩罚越大；lambda越大也是越希望获得结构简单的树。

## 对缺失值处理
通常情况下，人为处理缺失值的时候大多会选用中位数、均值或二者的融合来对数值型特征进行填补，使用出现次数多的类别来填补缺失的类别特征。  
在xgboost模型中允许缺失值存在。  

## 优缺点
优点：  
（1）xgBoosting支持线性分类器，相当于引入L1和L2正则化项的逻辑回归（分类问题）和线性回归（回归问题）；  
（2）xgBoosting对代价函数做了二阶Talor展开，引入了一阶导数和二阶导数；  
（3）当样本存在缺失值是，xgBoosting能自动学习分裂方向；  
（4）xgBoosting借鉴RF的做法，支持列抽样，这样不仅能防止过拟合，还能降低计算；  
（5）xgBoosting的代价函数引入正则化项，控制了模型的复杂度，正则化项包含全部叶子节点的个数，每个叶子节点输出的score的L2模的平方和。从贝叶斯方差角度考虑，正则项降低了模型的方差，防止模型过拟合；  
（6）xgBoosting在每次迭代之后，为叶子结点分配学习速率，降低每棵树的权重，减少每棵树的影响，为后面提供更好的学习空间；  
（7）xgBoosting工具支持并行,但并不是tree粒度上的，而是特征粒度，决策树最耗时的步骤是对特征的值排序，xgBoosting在迭代之前，先进行预排序，存为block结构，每次迭代，重复使用该结构，降低了模型的计算；block结构也为模型提供了并行可能，在进行结点的分裂时，计算每个特征的增益，选增益最大的特征进行下一步分裂，那么各个特征的增益可以开多线程进行；  
（8）可并行的近似直方图算法，树结点在进行分裂时，需要计算每个节点的增益，若数据量较大，对所有节点的特征进行排序，遍历的得到最优分割点，这种贪心法异常耗时，这时引进近似直方图算法，用于生成高效的分割点，即用分裂后的某种值减去分裂前的某种值，获得增益，为了限制树的增长，引入阈值，当增益大于阈值时，进行分裂；  

缺点：  
（1）xgBoosting采用预排序，在迭代之前，对结点的特征做预排序，遍历选择最优分割点，数据量大时，贪心法耗时，LightGBM方法采用histogram算法，占用的内存低，数据分割的复杂度更低；  
（2）xgBoosting采用level-wise生成决策树，同时分裂同一层的叶子，从而进行多线程优化，不容易过拟合，但很多叶子节点的分裂增益较低，没必要进行跟进一步的分裂，这就带来了不必要的开销；LightGBM采用深度优化，leaf-wise生长策略，每次从当前叶子中选择增益最大的结点进行分裂，循环迭代，但会生长出更深的决策树，产生过拟合，因此引入了一个阈值进行限制，防止过拟合.  

## sklearn参数
1、通用参数：宏观函数控制。 

2、Booster参数：控制每一步的booster(tree/regression)。 

3、学习目标参数：控制训练目标的表现。 

- XGBoost通用参数  

（1）booster（默认gbtree）：

         选择每次迭代的模型，有两种选择：gbtree（基于树的模型）和gbliner（线性模型）

（2）silent（默认0）：

        当这个参数值为1时，静默模式开启，不会输出任何信息。

        一般这个参数就保持默认的0，因为这样能帮我们更好地理解模型。

（3）nthread（默认为最大可能的线程数）：

        这个参数用来进行多线程控制，应当输入系统的核数。

        如果你希望使用CPU全部的核，那就不要输入这个参数，算法会自动检测它。

此外还有两个参数num_pbuffer和num_feature，由xgboost自动设置，用户不用管。

- Booster参数

前面提及有两种booster可选，但是实践表明tree的模型效果更好，linear很少用到。

（1）eta（默认0.3）：

        和GBDT中的 learning rate 参数类似。通过减少每一步的权重，可以提高模型的鲁棒性。

        典型值为0.01-0.2。

（2）min_child_weight（默认1）：

        决定最小叶子节点样本权重和。和GBDT的 min_child_leaf 参数类似，但不完全一样。

        XGBoost的这个参数是最小样本权重的和，而GBDT参数是最小样本总数。

        这个参数用于避免过拟合。当它的值较大时，可以避免模型学习到局部的特殊样本。

        但是如果这个值过高,会导致欠拟合。这个参数需要使用CV来调整。

（3）max_depth（默认6）：

        和GBM中的参数相同，这个值为树的最大深度。

        这个值也是用来避免过拟合的。max_depth越大，模型会学到更具体更局部的样本。

        需要使用CV函数来进行调优。

        典型值：3-10

（4）max_leaf_nodes：

        树上最大的节点或叶子的数量。

        可以替代max_depth的作用。因为如果生成的是二叉树，一个深度为n的树最多生成n方个叶子。

        如果定义了这个参数，GBM会忽略max_depth参数。

（5）gamma（默认0）：

        在节点分裂时，只有分裂后损失函数的值下降了，才会分裂这个节点。

        Gamma指定了节点分裂所需的最小损失函数下降值。

        这个参数的值越大，算法越保守。这个参数的值和损失函数息息相关，所以是需要调整的

（6）max_delta_step（默认0）：

        这参数限制每棵树权重改变的最大步长。

        如果这个参数的值为0，那就意味着没有约束。如果它被赋予了某个正值，那么它会让这个算法更加保守。

        通常，这个参数不需要设置。但是当各类别的样本十分不平衡时，它对逻辑回归是很有帮助的。

        这个参数一般用不到，但是你可以挖掘出来它更多的用处。

（7）subsample（默认1）：

        和GBM中的subsample参数一模一样。这个参数控制对于每棵树，随机采样的比例。

        减小这个参数的值，算法会更加保守，避免过拟合。但是，如果这个值设置得过小，它可能会导致欠拟合。

        典型值：0.5-1

（8）colsample_bylevel（默认1）：

        用来控制树的每一级的每一次分裂，对列数的采样的占比。

        典型值：0.5-1

（9）colsample_bytree（默认1）：

        和GBM里面的max_features参数类似。用来控制每棵随机采样的列数的占比(每一列是一个特征)

        典型值：0.5-1

（10）lambda（默认1）：

        权重的L2正则化项。(和Ridge regression类似)。

        这个参数是用来控制XGBoost的正则化部分的，增大这个值会使得模型更为保守

（11）alpha（默认0）：

        权重的L1正则化项。(和Lasso regression类似)，增大这个值会使得模型更为保守

        可以应用在很高维度的情况下，使得算法的速度更快。

（12）scale_pos_weight（默认1）：

        在各类别样本十分不平衡时，把这个参数设定为一个正值，可以使算法更快收敛。


- 学习目标参数

（1）objective（默认reg：linear）：

        这个参数定义需要被最小化的损失函数。最常用的值有： 
            binary:logistic 二分类的逻辑回归，返回预测的概率(不是类别)。
            multi:softmax 使用softmax的多分类器，返回预测的类别(不是概率)。 
                在这种情况下，你还需要多设一个参数：num_class(类别数目)。
           multi:softprob 和multi:softmax参数一样，但是返回的是每个数据属于各个类别的概率。

（2）eval_metric（默认值取决于objective参数的取值）：

        对于有效数据的度量方法。
        对于回归问题，默认值是rmse，对于分类问题，默认值是error。
        典型值有： 
            rmse 均方根误差
            mae 平均绝对误差
            logloss 负对数似然函数值
            error 二分类错误率(阈值为0.5)
            merror 多分类错误率
            mlogloss 多分类logloss损失函数
            auc 曲线下面积

（3）seed（默认0）：

        随机数种子。设置它可以复现随机数据的结果，也可以用于调整参数

## 应用场景

有监督学习，分类、回归皆可以

## Reference  
https://blog.csdn.net/Kaiyuan_sjtu/article/details/80018580  
https://blog.csdn.net/Guiabbey/article/details/89085101    
https://nbviewer.jupyter.org/github/huanwn111/test/blob/master/whale06_gj_day3.ipynb  